## Var declarations

In [16]:
data_file_path = "/Users/lorenapacheco/Concordia/Masters/BugReportsMining/data/merged_data_production_bug_reports.json"

## General methods

In [17]:
import json
import os

def json_file_to_dict(file):
    data = {}
    with open(os.path.join(file), 'r') as fp:
        data = json.load(fp)
    fp.close()
    return data

def dict_to_json_file(file, dic):
    folder = os.path.dirname(file)
    if not os.path.exists(folder):
        os.makedirs(folder)
    with open(os.path.join(file), 'w') as fp:
        json.dump(dic, fp, sort_keys=True, indent=4)
    fp.close()

In [25]:
import re

def extract_info_from_stack_trace(stack_trace_entry):
    file_name = None
    method_name = None
    file_line = -1

    match = re.search(r'\s*at\s+([\w.$<>]+)\(([^:]*\.java):(\d+)\)', stack_trace_entry)
    if match:
        method_name = match.group(1)
        file_name = match.group(2)
        file_line = int(match.group(3))

    return file_name, method_name, file_line

bugs_data = json_file_to_dict(data_file_path)
for project in bugs_data.keys():
    for bug_id in bugs_data[project].keys():
        stack_trace_entries = []
        for stack_trace in bugs_data[project][bug_id]["log"]:
            stack_trace_entries += stack_trace.replace("\t", "").split("\n")
        st_methods = []
        st_files = []
        st_lines = []
        for st_entry in stack_trace_entries:
            try:
                file_name, method_name, line_number = extract_info_from_stack_trace(st_entry)
                if file_name is not None and method_name is not None and line_number is not None:
                    st_methods.append(method_name)
                    st_files.append(file_name)
                    st_lines.append(str(line_number))
            except:
                continue
        bugs_data[project][bug_id]["stack_trace_methods"] = st_methods
        bugs_data[project][bug_id]["stack_trace_files"] = st_files
        bugs_data[project][bug_id]["stack_trace_lines"] = st_lines
dict_to_json_file(data_file_path,bugs_data)
print("Done")

Done
